<a href="https://colab.research.google.com/github/mmadduri/BMI_Model/blob/master/20200810_learning_rate_bmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# "magic" commands, prefaced with "%", changes settings in the notebook

# this ensures plots are embedded in notebook web page
%matplotlib inline

# pdb = Python debugger, so this command turns the debugger OFF
%pdb off

# numpy = numerical Python, implements arrays (/ matrices)
import numpy as np
# limit number of decimal places printed for floating-point numbers
np.set_printoptions(precision=3)

# scipy = scientific Python, implements operations on arrays / matrices
import scipy as sp
# linalg = linear algebra, implements eigenvalues, matrix inverse, etc
from scipy import linalg as la
# optimize = optimization, root finding, etc
from scipy import optimize as op

# produce matlab-style plots
import matplotlib as mpl
# increase font size on plots
mpl.rc('font',**{'size':18})
# use LaTeX to render symbols
mpl.rc('text',usetex=False)
# animation
from matplotlib import animation as ani
# Matlab-style plotting
import matplotlib.pyplot as plt

# symbolic computation, i.e. computer algebra (like Mathematica, Wolfram Alpha)
import sympy as sym

Automatic pdb calling has been turned OFF


In [ ]:
# test whether this is a Colaboratory or Jupyter notebook
try:
  import google.colab
  COLAB = True
  print('Colaboratory Notebook')
except:
  COLAB = False
  print('Jupyter Notebook')

# Colab notebook
if COLAB:  
  # render SymPy equations nicely in Colaboratory Notebook
  def colab_latex_printer(exp,**options):
    from google.colab.output._publish import javascript
    url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
    javascript(url=url)
    return sym.printing.latex(exp,**options)
  
  sym.init_printing(use_latex="mathjax",latex_printer=colab_latex_printer)

# Jupyter notebook
else:
  init_printing(use_latex='mathjax')

Colaboratory Notebook


In [ ]:
import sympy as sym
from sympy import symbols, diff, Matrix, solve

In [ ]:
k, w, tau, gam, gam_k, gam_w, sig_k, sig_w, sig, alpha = symbols(r'k w \tau \gamma \gamma_k \gamma_w \sigma_k \sigma_w \sigma \alpha')
k,w,tau,gam,gam_k,gam_w,sig_k, sig_w, sig, alpha

(k, w, \tau, \gamma, \gammaₖ, \gamma_w, \sigmaₖ, \sigma_w, \sigma, \alpha)

In [ ]:
sig,sig_k,sig_w = sym.symbols(r'\sigma \sigma_k \sigma_w',nonnegative=True)

In [ ]:
e_scl = ( ((tau**2)*(1 - k*w)**2) + (sig_w * w**2) + (sig_k*k**2)) / 2
# sig_w and sig_k are penalty parameters
# 1/2 term added to make the derivative simpler 

In [ ]:
e_scl

         2             2       2           2
\sigmaₖ⋅k    \sigma_w⋅w    \tau ⋅(-k⋅w + 1) 
────────── + ─────────── + ─────────────────
    2             2                2        

In [ ]:
# First Derivative
de_dk = diff(e_scl,k)
de_dw = diff(e_scl, w)

# Second Derivative
de2_d2k = diff(de_dk, k)
de2_dwdk = diff(de_dk, w)
de2_d2w = diff(de_dw, w)
de2_dkdw = diff(de_dw, k)

In [ ]:
de = [de_dk, de_dw]
de

⎡                2                                2             ⎤
⎣\sigmaₖ⋅k - \tau ⋅w⋅(-k⋅w + 1), \sigma_w⋅w - \tau ⋅k⋅(-k⋅w + 1)⎦

Continuous-time gradient descent has dynamics:

$$ \dot{x} = - De(x) = f(x), $$

So the Jacobian of the dynamics is determined by the Hessian of error $e$:

$$ Df(x) = - D^2 e(x). $$

At a minimum $x_0$, the eigenvalues of $D^2 e(x_0)$ are positive, so the eigenvalues of $Df(x_0)$ are negative.

Note: The Hessian matrix of a function f is the Jacobian matrix of the gradient of the function: H(f(x)) = J(∇f(x)). (https://en.wikipedia.org/wiki/Hessian_matrix)



In [ ]:
Gamma = sym.Matrix.diag([1,1])
Gamma

⎡1  0⎤
⎢    ⎥
⎣0  1⎦

In [ ]:
# sets f(x) = -De(x)
f = -Gamma*sym.Matrix(de)
f

⎡                 2              ⎤
⎢-\sigmaₖ⋅k + \tau ⋅w⋅(-k⋅w + 1) ⎥
⎢                                ⎥
⎢                  2             ⎥
⎣-\sigma_w⋅w + \tau ⋅k⋅(-k⋅w + 1)⎦

In [ ]:
# substitutes tau = 1, k_sig = w_sig = sig
# subs = {tau:1,sig_k:sig,sig_w:sig}
subs = {sig_k:sig, sig_w:1}

In [ ]:
# stationary points 
# these stationary points lead to points where e(k_0, w_0) is a minimum, maximum or saddle point
# Note: without the penalty terms in the cost function, (0,0) becomes a saddle point as a stationary point
sol = solve(de,[k,w]) # <-- find [k_0,w_0] that makes de(k_0,w_0) == [0,0]
sol

⎡        ⎛                                                             _______
⎢        ⎜          ⎛                ⎛    _________           ⎞⎞      ╱     __
⎢        ⎜          ⎜              2 ⎜  ╲╱ \sigmaₖ     \sigmaₖ⎟⎟     ╱    ╲╱ \
⎢        ⎜-\sigma_w⋅⎜\sigmaₖ + \tau ⋅⎜- ──────────── - ───────⎟⎟⋅   ╱   - ────
⎢        ⎜          ⎜                ⎜    __________        2 ⎟⎟   ╱        __
⎢        ⎜          ⎝                ⎝  ╲╱ \sigma_w     \tau  ⎠⎠ ╲╱       ╲╱ \
⎢(0, 0), ⎜────────────────────────────────────────────────────────────────────
⎢        ⎜                                                 2                  
⎣        ⎝                                     \sigmaₖ⋅\tau                   

___________________                                    ⎞  ⎛                   
_______                                                ⎟  ⎜         ⎛         
sigmaₖ     \sigmaₖ                                     ⎟  ⎜         ⎜         
──────── - ───────           ______________________

In [ ]:
# substitutes subs for the fixed points
# 2 are imaginary -- ignored
# other 2 are real if the sigma < 1 -- sigma can't be too large
# if sigma is too large, going back to (0, 0)
[sym.simplify(sym.Matrix(_).subs(subs)) for _ in sol]

⎡     ⎡     _______________________ ⎤  ⎡      _______________________ ⎤  ⎡    
⎢     ⎢    ╱     ________   \sigma  ⎥  ⎢     ╱     ________   \sigma  ⎥  ⎢    
⎢     ⎢   ╱  - ╲╱ \sigma  - ──────  ⎥  ⎢-   ╱  - ╲╱ \sigma  - ──────  ⎥  ⎢-   
⎢     ⎢  ╱                      2   ⎥  ⎢   ╱                      2   ⎥  ⎢   ╱
⎢     ⎢╲╱                   \tau    ⎥  ⎢ ╲╱                   \tau    ⎥  ⎢ ╲╱ 
⎢     ⎢──────────────────────────── ⎥  ⎢──────────────────────────────⎥  ⎢────
⎢⎡0⎤  ⎢           ________          ⎥  ⎢            ________          ⎥  ⎢    
⎢⎢ ⎥, ⎢         ╲╱ \sigma           ⎥, ⎢          ╲╱ \sigma           ⎥, ⎢    
⎢⎣0⎦  ⎢                             ⎥  ⎢                              ⎥  ⎢    
⎢     ⎢      _______________________⎥  ⎢      _______________________ ⎥  ⎢    
⎢     ⎢     ╱     ________   \sigma ⎥  ⎢     ╱     ________   \sigma  ⎥  ⎢    
⎢     ⎢-   ╱  - ╲╱ \sigma  - ────── ⎥  ⎢    ╱  - ╲╱ \sigma  - ──────  ⎥  ⎢-   
⎢     ⎢   ╱                      2  ⎥  ⎢   ╱        

In [ ]:
# Separating the substitutions -- this is if tau = 1 only 
[sym.simplify(sym.expand(sym.simplify(sym.Matrix(_).subs({tau:1})))) for _ in sol]

⎡     ⎡                ______________________________________⎤  ⎡             
⎢     ⎢4 __________   ╱     _________             __________ ⎥  ⎢   4 ________
⎢     ⎢╲╱ \sigma_w ⋅╲╱  - ╲╱ \sigmaₖ  - \sigmaₖ⋅╲╱ \sigma_w  ⎥  ⎢-ⅈ⋅╲╱ \sigma_
⎢     ⎢──────────────────────────────────────────────────────⎥  ⎢─────────────
⎢     ⎢                       _________                      ⎥  ⎢             
⎢⎡0⎤  ⎢                     ╲╱ \sigmaₖ                       ⎥  ⎢             
⎢⎢ ⎥, ⎢                                                      ⎥, ⎢             
⎢⎣0⎦  ⎢                 __________________________           ⎥  ⎢             
⎢     ⎢                ╱     _________                       ⎥  ⎢             
⎢     ⎢               ╱    ╲╱ \sigmaₖ                        ⎥  ⎢             
⎢     ⎢          -   ╱   - ──────────── - \sigmaₖ            ⎥  ⎢             
⎢     ⎢             ╱        __________                      ⎥  ⎢             
⎣     ⎣           ╲╱       ╲╱ \sigma_w              

In [ ]:
# expands de with the substitutions above
sym.expand(sym.simplify(sym.Matrix(de).subs(subs)))

⎡               2    2       2  ⎤
⎢\sigma⋅k + \tau ⋅k⋅w  - \tau ⋅w⎥
⎢                               ⎥
⎢               2  2         2  ⎥
⎣\sigma⋅w + \tau ⋅k ⋅w - \tau ⋅k⎦

In [ ]:
# Looking at one of the solved fixed points
k0,w0 = sym.simplify(sym.Matrix(sol[3]).subs(subs))
x0 = {k:k0,w:w0}
x0

⎧         ______________           ______________⎫
⎪        ╱   \sigma               ╱   \sigma     ⎪
⎨k: -   ╱  - ────── + 1 , w: -   ╱  - ────── + 1 ⎬
⎪      ╱         2              ╱         2      ⎪
⎩    ╲╱      \tau             ╲╱      \tau       ⎭

In [ ]:
H = sym.Matrix(de).jacobian([k,w])
J = f.jacobian([k,w])

In [ ]:
H0 = H.subs(subs).subs(x0)
J0 = J.subs(subs).subs(x0)

In [ ]:
J0.subs({sig:sym.Rational(1,2)}).eigenvals()
J0.eigenvals()

⎧                               2   ⎫
⎨-2⋅\sigma: 1, 2⋅\sigma - 2⋅\tau : 1⎬
⎩                                   ⎭

In [ ]:
H0.eigenvals()

⎧                               2   ⎫
⎨2⋅\sigma: 1, -2⋅\sigma + 2⋅\tau : 1⎬
⎩                                   ⎭

When it comes to $J = Df(x_0)$, we care about whether all eigenvalues have negative real part,

$$ \forall \lambda \in \operatorname{spec} Df(x_0) : \operatorname{Real}\lambda < 0, $$

because we have the bound 

$$ \| x(t) - x_0 \| \leq e^{\overline{\lambda} t} \| x(0) - x_0 \| $$

where 

$$ \overline{\lambda} = \max\{\operatorname{Real}\lambda : \lambda\in \operatorname{spec} Df(x_0)\}. $$

Continuous-time gradient descent has dynamics:

$$ \dot{x} = - De(x) = f(x), $$

So on time horizon $\Delta > 0$, we have approximately

$$ x(t + \Delta) - x(t) \approx \Delta f(x(t)). $$

Discrete-time gradient descent has dynamics:

$$ x^+ = x - \Gamma De(x) = x + \Gamma f(x) = F(x), $$

Note that if $x_0$ is stationary:

$$ De(x_0) = 0 \implies f(x_0) = 0 \implies \dot{x}_0 = 0 $$

$$ De(x_0) = 0 \implies F(x_0) = x_0 $$

In [ ]:
Gamma = sym.Matrix.diag([gam_k,gam_w])
Gamma

⎡\gammaₖ     0    ⎤
⎢                 ⎥
⎣   0     \gamma_w⎦

In [ ]:
# Set F as the matrix to describe the update to k+, w+
F = sym.Matrix([k,w]) + Gamma * f.subs(subs)
F

⎡        ⎛                2             ⎞     ⎤
⎢\gammaₖ⋅⎝-\sigma⋅k + \tau ⋅w⋅(-k⋅w + 1)⎠ + k ⎥
⎢                                             ⎥
⎢         ⎛                2             ⎞    ⎥
⎣\gamma_w⋅⎝-\sigma⋅w + \tau ⋅k⋅(-k⋅w + 1)⎠ + w⎦

In [ ]:
# Discrete time, derivative of F
DF = F.jacobian([k,w])
DF

⎡            ⎛              2  2⎞                   ⎛      2           2      
⎢    \gammaₖ⋅⎝-\sigma - \tau ⋅w ⎠ + 1       \gammaₖ⋅⎝- \tau ⋅k⋅w + \tau ⋅(-k⋅w
⎢                                                                             
⎢         ⎛      2           2           ⎞              ⎛              2  2⎞  
⎣\gamma_w⋅⎝- \tau ⋅k⋅w + \tau ⋅(-k⋅w + 1)⎠     \gamma_w⋅⎝-\sigma - \tau ⋅k ⎠ +

     ⎞⎤
 + 1)⎠⎥
      ⎥
      ⎥
 1    ⎦

In [ ]:
# Sub in the stationary point for the eignenvalues
DF.subs(x0).eigenvals()

⎧                                      _______________________________________
⎪              2                2     ╱        2     4                        
⎨  \gammaₖ⋅\tau    \gamma_w⋅\tau    ╲╱  \gammaₖ ⋅\tau  + 16⋅\gammaₖ⋅\gamma_w⋅\
⎪- ───────────── - ────────────── - ──────────────────────────────────────────
⎩        2               2                                                    

______________________________________________________________________________
     2                                  2                          4          
sigma  - 16⋅\gammaₖ⋅\gamma_w⋅\sigma⋅\tau  + 2⋅\gammaₖ⋅\gamma_w⋅\tau  + \gamma_
──────────────────────────────────────────────────────────────────────────────
                      2                                                       

_________                                               ______________________
 2     4                        2                2     ╱        2     4       
w ⋅\tau             \gammaₖ⋅\tau    \gamma_w⋅\tau 

When it comes to $DF(x_0)$, we care about whether all eigenvalues have magnitude smaller than $1$,

$$ \forall \lambda \in \operatorname{spec} DF(x_0) : |\lambda| < 1, $$

because we have the bound 

$$ \| x(k) - x_0 \| \leq \widetilde{\lambda}^k \| x(0) - x_0 \| $$

where 

$$ \widetilde{\lambda} = \max\{|\lambda| : \lambda\in \operatorname{spec} DF(x_0)\}. $$

In [ ]:
# substitution of sigma = 1/2
# iterative gradient descent converges?
# Two-Learner paper connection: learning rates can't be too large or small
DF.subs(x0).subs({sig:sym.Rational(1,2), gam_k:gam, gam_w:gam}).eigenvals()
DF.subs(x0).subs({sig:sym.Rational(1,2)}).eigenvals()

⎧                                      _______________________________________
⎪              2                2     ╱        2     4                        
⎨  \gammaₖ⋅\tau    \gamma_w⋅\tau    ╲╱  \gammaₖ ⋅\tau  + 2⋅\gammaₖ⋅\gamma_w⋅\t
⎪- ───────────── - ────────────── - ──────────────────────────────────────────
⎩        2               2                                                    

______________________________________________________________________        
  4                          2                                2     4         
au  - 8⋅\gammaₖ⋅\gamma_w⋅\tau  + 4⋅\gammaₖ⋅\gamma_w + \gamma_w ⋅\tau          
────────────────────────────────────────────────────────────────────── + 1: 1,
             2                                                                

                                       _______________________________________
               2                2     ╱        2     4                        
   \gammaₖ⋅\tau    \gamma_w⋅\tau    ╲╱  \gammaₖ ⋅\

In [ ]:
DF.subs(x0).eigenvals()

⎧                                      _______________________________________
⎪              2                2     ╱        2     4                        
⎨  \gammaₖ⋅\tau    \gamma_w⋅\tau    ╲╱  \gammaₖ ⋅\tau  + 16⋅\gammaₖ⋅\gamma_w⋅\
⎪- ───────────── - ────────────── - ──────────────────────────────────────────
⎩        2               2                                                    

______________________________________________________________________________
     2                                  2                          4          
sigma  - 16⋅\gammaₖ⋅\gamma_w⋅\sigma⋅\tau  + 2⋅\gammaₖ⋅\gamma_w⋅\tau  + \gamma_
──────────────────────────────────────────────────────────────────────────────
                      2                                                       

_________                                               ______________________
 2     4                        2                2     ╱        2     4       
w ⋅\tau             \gammaₖ⋅\tau    \gamma_w⋅\tau 

In [ ]:
# As in the block of text above, the eigenvalue that has the larger value is the one to pay attention to
# Take the log of ||KW - K_oW_o|| closer to the stationary point 
# Compare that with the t*log(eigenvalue) to get the convergence rate
DF.subs(x0).subs({sig:sym.Rational(1,2), tau:1}).eigenvals()

{-\gammaₖ + 1: 1, -\gamma_w + 1: 1}

In [ ]:
F_num = sym.lambdify([k,w],F.subs({sig:sym.Rational(1,2),gam_k:2.001,gam_w:.9}))
x = [np.random.randn(2)]
for i in range(100):
  x.append(F_num(*x[-1]).flatten())
x = np.asarray(x)
plt.plot(x)

NameError: ignored

# ^^^ Sam messed this up

###Trying above as a vector

In [ ]:
from sympy import Identity,MatrixSymbol
from sympy.abc import i, j, k, l, N
W = MatrixSymbol("W", N, 1)
K = MatrixSymbol("K", N, 1)
I1 = Identity(1)

In [ ]:
dcv_dK = -(tau**2)*(I1 - K.T*W)*W.T + sig_k*K.T
dcv_dW = -(tau**2)*(I1 - K.T*W)*K.T + sig_w*W.T

In [ ]:
[dcv_dK, dcv_dW]

⎡         T        2 ⎛      T  ⎞  T            T        2 ⎛      T  ⎞  T⎤
⎣\sigmaₖ⋅K  + -\tau ⋅⎝I + -K ⋅W⎠⋅W , \sigma_w⋅W  + -\tau ⋅⎝I + -K ⋅W⎠⋅K ⎦

can tell that K and W are co-linear at the stationary points, so can say W = alpha*K and can assume that sigma_k = sigma_w

In [ ]:
subs = {tau:1,sig_k:sig,sig_w:sig}


In [ ]:
dcv_dK = dcv_dK.subs(subs)
dcv_dW = dcv_dW.subs(subs)

In [ ]:
[dcv_dK, dcv_dW]

⎡        T    ⎛      T  ⎞  T          T    ⎛      T  ⎞  T⎤
⎣\sigma⋅K  + -⎝I + -K ⋅W⎠⋅W , \sigma⋅W  + -⎝I + -K ⋅W⎠⋅K ⎦

In [ ]:
subs2 = {W:alpha*K}
dcv_dK = dcv_dK.subs(subs).subs(subs2)
dcv_dW = dcv_dW.subs(subs).subs(subs2)

In [ ]:
[dcv_dK, dcv_dW]

⎡        T    ⎛      T         ⎞           T                   T    ⎛      T  
⎣\sigma⋅K  + -⎝I + -K ⋅\alpha⋅K⎠⋅(\alpha⋅K) , \sigma⋅(\alpha⋅K)  + -⎝I + -K ⋅\

       ⎞  T⎤
alpha⋅K⎠⋅K ⎦

Setting both derivatives to 0 to find stationary points

In [ ]:
dcv_dK 
sym.solve(dcv_dK, alpha)

[]

### Scratch Work

In [ ]:
[[de2_d2k, de2_dwdk], [de2_dkdw, de2_d2w]]

⎡⎡              2  2      2           2           ⎤  ⎡    2           2       
⎣⎣\sigmaₖ + \tau ⋅w , \tau ⋅k⋅w - \tau ⋅(-k⋅w + 1)⎦, ⎣\tau ⋅k⋅w - \tau ⋅(-k⋅w 

                     2  2⎤⎤
+ 1), \sigma_w + \tau ⋅k ⎦⎦

In [ ]:
# Jacobian Matrix
Jcb_e_scl = Matrix([de_dk, de_dw]).jacobian([k,w])
Jcb_e_scl

⎡                   2  2           2           2           ⎤
⎢     \sigmaₖ + \tau ⋅w        \tau ⋅k⋅w - \tau ⋅(-k⋅w + 1)⎥
⎢                                                          ⎥
⎢    2           2                                2  2     ⎥
⎣\tau ⋅k⋅w - \tau ⋅(-k⋅w + 1)      \sigma_w + \tau ⋅k      ⎦

In [ ]:
# Evals of jacobian
Jcb_e_scl.eigenvals()

⎧                                              _______________________________
⎪                         2  2       2  2     ╱        2                      
⎨\sigmaₖ   \sigma_w   \tau ⋅k    \tau ⋅w    ╲╱  \sigmaₖ  - 2⋅\sigmaₖ⋅\sigma_w 
⎪─────── + ──────── + ──────── + ──────── - ──────────────────────────────────
⎩   2         2          2          2                                         

______________________________________________________________________________
                2  2                 2  2           2                  2  2   
- 2⋅\sigmaₖ⋅\tau ⋅k  + 2⋅\sigmaₖ⋅\tau ⋅w  + \sigma_w  + 2⋅\sigma_w⋅\tau ⋅k  - 
──────────────────────────────────────────────────────────────────────────────
                                                               2              

______________________________________________________________________________
               2  2       4  4          4  2  2          4           4  4     
2⋅\sigma_w⋅\tau ⋅w  + \tau ⋅k  + 14⋅\tau ⋅k ⋅w  - 

In [ ]:
Jcb_e_scl.eigenvals().keys()

dict_keys([\sigma_k/2 + \sigma_w/2 + \tau**2*k**2/2 + \tau**2*w**2/2 - sqrt(\sigma_k**2 - 2*\sigma_k*\sigma_w - 2*\sigma_k*\tau**2*k**2 + 2*\sigma_k*\tau**2*w**2 + \sigma_w**2 + 2*\sigma_w*\tau**2*k**2 - 2*\sigma_w*\tau**2*w**2 + \tau**4*k**4 + 14*\tau**4*k**2*w**2 - 16*\tau**4*k*w + \tau**4*w**4 + 4*\tau**4)/2, \sigma_k/2 + \sigma_w/2 + \tau**2*k**2/2 + \tau**2*w**2/2 + sqrt(\sigma_k**2 - 2*\sigma_k*\sigma_w - 2*\sigma_k*\tau**2*k**2 + 2*\sigma_k*\tau**2*w**2 + \sigma_w**2 + 2*\sigma_w*\tau**2*k**2 - 2*\sigma_w*\tau**2*w**2 + \tau**4*k**4 + 14*\tau**4*k**2*w**2 - 16*\tau**4*k*w + \tau**4*w**4 + 4*\tau**4)/2])

In [ ]:
sym.solve(diff(e_scl,k),k)

⎡         2        ⎤
⎢     \tau ⋅w      ⎥
⎢──────────────────⎥
⎢              2  2⎥
⎣\sigmaₖ + \tau ⋅w ⎦

In [ ]:
Matrix([[k, w],[-w, k]]).eigenvals()
Matrix([[0, 1],[-2, 3]]).eigenvals()

{1: 1, 2: 1}

In [ ]:
sol = sym.solve([diff(e_scl,k),diff(e_scl,w)],[k,w])
sol

⎡        ⎛                                                             _______
⎢        ⎜          ⎛                ⎛    _________           ⎞⎞      ╱     __
⎢        ⎜          ⎜              2 ⎜  ╲╱ \sigmaₖ     \sigmaₖ⎟⎟     ╱    ╲╱ \
⎢        ⎜-\sigma_w⋅⎜\sigmaₖ + \tau ⋅⎜- ──────────── - ───────⎟⎟⋅   ╱   - ────
⎢        ⎜          ⎜                ⎜    __________        2 ⎟⎟   ╱        __
⎢        ⎜          ⎝                ⎝  ╲╱ \sigma_w     \tau  ⎠⎠ ╲╱       ╲╱ \
⎢(0, 0), ⎜────────────────────────────────────────────────────────────────────
⎢        ⎜                                                 2                  
⎣        ⎝                                     \sigmaₖ⋅\tau                   

___________________                                    ⎞  ⎛                   
_______                                                ⎟  ⎜         ⎛         
sigmaₖ     \sigmaₖ                                     ⎟  ⎜         ⎜         
──────── - ───────           ______________________

In [ ]:
sol[k]

TypeError: ignored

In [ ]:
e_scl.subs(sol)

In [ ]:
e_scl.subs({k:sol[k]})

In [ ]:
e_scl.subs({sig_k:sig,sig_w:sig,tau:1})

In [ ]:
sym.diff?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def g_w(w,k):
  return w - 0.001*w - 0.1*w*k
def g_k(w,k):
  return k - 0.01*k +0.1*w*k

w0 = 1.
k0 = .5
max_iter = int(1E3)

w_ = np.zeros(max_iter) 
k_ = np.zeros(max_iter) 
w_[0] = w0
k_[0] = k0
for t in range(max_iter-1):
  k_[t+1] = g_k(w_[t],k_[t])
  w_[t+1] = g_w(w_[t],k_[t])

plt.plot(w_)
plt.plot(k_)


In [ ]:
w_